(target-dj-querying-data)=
# DataJoint pipeline: Querying data

:::{important}
Before you begin, ensure you have a [DataJoint pipeline deployed](target-dj-pipeline-deployment) and have [data ingested into the pipeline](target-dj-data-ingestion-processing).
:::

This notebook provides a few examples on how to query data from the [Aeon DataJoint pipeline](target-aeon-dj-pipeline). 

:::{note}
This notebook uses the [Single mouse in a foraging assay](sample-data-single-mouse-foraging:) sample dataset for the experiment named `social0.2-aeon3`. 
If you are using a different dataset, please replace the experiment name and other parameters in the code below accordingly.
:::

In [ ]:
from aeon.dj_pipeline import acquisition, tracking
from aeon.dj_pipeline.analysis import block_analysis


To query the entire `acquisition.Chunk` table:

In [ ]:
acquisition.Chunk()

With different [operators](datajoint:docs/core/datajoint-python/0.14/query/operators/), we can perform complex queries and manipulations on the data,
enabling us to filter, join, project, aggregate, and manipulate data in the DataJoint tables.
For example, we can use the [restriction operator `&`](datajoint:docs/core/datajoint-python/0.14/query/operators/#restriction) to retrieve all {term}`chunks <Acquisition Chunk>` associated with the experiment `social0.2-aeon3`:

In [ ]:
experiment_key = {"experiment_name": "social0.2-aeon3"}
acquisition.Chunk & experiment_key


To query position data from [SLEAP](sleap:) tracking:

In [ ]:
tracking.SLEAPTracking & experiment_key

In [ ]:
tracking.SLEAPTracking.PoseIdentity & experiment_key

In [ ]:
tracking.SLEAPTracking.Part & experiment_key

To query all experiment `social0.2-aeon3` _blocks_ and analyses at the _block_ level:

In [ ]:
block_analysis.Block & experiment_key

In [ ]:
block_analysis.BlockAnalysis & experiment_key

To [`fetch()`](datajoint:docs/core/datajoint-python/0.14/query/fetch/#fetch) the [primary keys](datajoint:docs/core/datajoint-python/0.14/design/tables/primary/) of all experiment `social0.2-aeon3` _blocks_:

In [ ]:
block_keys = (block_analysis.Block & experiment_key).fetch("KEY")

Let's pick a random block key to inspect further the block analysis tables:

In [ ]:
block_key = block_keys[10]

In [ ]:
block_analysis.BlockAnalysis.Patch & block_key

In [ ]:
block_analysis.BlockAnalysis.Subject & block_key

In [ ]:
block_analysis.BlockSubjectAnalysis.Patch & block_key

In [ ]:
block_analysis.BlockSubjectAnalysis.Preference & block_key

In [ ]:
# subject position
block_analysis.BlockSubjectPositionPlots.InROI & block_key


In [ ]:
inroi_df = (
    (block_analysis.BlockSubjectPositionPlots.InROI & block_key)
    .fetch(format="frame")
    .reset_index()
)
inroi_df
